# 準備

## artのインストール

In [1]:
!pip3 install adversarial-robustness-toolbox

     |████████████████████████████████| 1.1 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 32.1 MB/s 
     |████████████████████████████████| 26.3 MB 3.7 MB/s 
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Attempting uninstall: numba
    Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


## ライブラリのインポート

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

# TensorFlow with Keras.
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.layers import MaxPooling2D, GlobalAveragePooling2D, Dropout
tf.compat.v1.disable_eager_execution()

# ART
import art
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier

## CIFAR10のロード・前処理

In [2]:
# CIFAR10のロード。
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

# CIFAR10のラベル。
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
num_classes = len(classes)

In [3]:
# 正規化。
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

# ラベルをOne-hot-vector化。
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

## 摂動を作るためのClassifierの作成



### モデル定義

In [4]:
# モデルの定義。
def build_model():
  inputs = Input(shape=(32, 32, 3))
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(inputs)
  x = Conv2D(64, (3, 3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(128, (3,3), padding='SAME', activation='relu')(x)
  x = Dropout(0.25)(x)
  x = MaxPooling2D()(x)

  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = Conv2D(256, (3,3), padding='SAME', activation='relu')(x)
  x = GlobalAveragePooling2D()(x)

  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.25)(x)
  y = Dense(10, activation='softmax')(x)

  model = Model(inputs, y)

  model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

  return model

model = build_model()

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584

### 学習の実行

In [5]:
# 学習の実行。
model.fit(X_train, y_train,
          batch_size=512,
          epochs=30,
          validation_data=(X_test, y_test),
          shuffle=True)

Train on 50000 samples, validate on 10000 samples


2021-12-16 03:05:55.190384: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/30
 8192/50000 [===>..........................] - ETA: 4:20 - loss: 2.2998 - accuracy: 0.1124

KeyboardInterrupt: 

### モデルの精度評価

In [ ]:
model.evaluate(X_test, y_test)

[0.6143605302333832, 0.8239]

## Assistive Signalsの作成

In [ ]:
# 入力データの特徴量の最小値・最大値を指定。
# 特徴量は0.0～1.0の範囲に収まるように正規化しているため、最小値は0.0、最大値は1.0とする。
min_pixel_value = 0.0
max_pixel_value = 1.0

# モデルをART Keras Classifierでラップ。
classifier = KerasClassifier(model=model, clip_values=(min_pixel_value, max_pixel_value), use_logits=False)

In [ ]:
# FGSMインスタンスの作成。
attack_targeted = FastGradientMethod(estimator=classifier, eps=0.10, targeted=True)
# 敵対的サンプルの生成（ベース画像はテストデータとする）。
X_test_assistive = attack_targeted.generate(x=X_test, y=y_test)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


## 非標的型攻撃としてのFGSMによる摂動を作成
以下、この摂動を上乗せした画像のことを「敵対的画像」と呼ぶ。

In [ ]:
attack_notargeted = FastGradientMethod(estimator=classifier, eps=0.10, targeted=False)
X_test_adv = attack_notargeted.generate(X_test)

## 敵対的画像に対するモデルの精度評価

### cleanのみで学習したモデルとAssistive Signals

In [ ]:
model.evaluate(X_test_assistive, y_test)

[1.495119759941101, 0.6596]

### cleanのみで学習したモデルと敵対的攻撃画像

In [ ]:
model.evaluate(X_test_adv, y_test)

[7.404366830444336, 0.0955]

# Assistive Signals入りの画像のみで学習

In [ ]:
X_train_assistive = attack_targeted.generate(x=X_train, y=y_train)

In [ ]:
model2 = build_model()

In [ ]:
model2.fit(X_train_assistive, y_train,
          batch_size=512,
          epochs=30,
          shuffle=True)

Train on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 31s 611us/sample - loss: 2.0327 - accuracy: 0.2237
Epoch 2/30
50000/50000 [==============================] - 30s 608us/sample - loss: 1.4930 - accuracy: 0.4424
Epoch 3/30
50000/50000 [==============================] - 30s 607us/sample - loss: 0.9644 - accuracy: 0.6611
Epoch 4/30
50000/50000 [==============================] - 30s 608us/sample - loss: 0.6036 - accuracy: 0.7965
Epoch 5/30
50000/50000 [==============================] - 30s 607us/sample - loss: 0.3913 - accuracy: 0.8697
Epoch 6/30
50000/50000 [==============================] - 30s 607us/sample - loss: 0.2631 - accuracy: 0.9146
Epoch 7/30
50000/50000 [==============================] - 30s 607us/sample - loss: 0.1904 - accuracy: 0.9364
Epoch 8/30
50000/50000 [==============================] - 31s 611us/sample - loss: 0.1458 - accuracy: 0.9515
Epoch 9/30
50000/50000 [==============================] - 31s 616us/sample - loss: 0.1012 - accuracy: 0.9

In [ ]:
# assistive train data
model2.evaluate(X_train_assistive, y_train)

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


[0.008268624045625911, 0.99768]

In [ ]:
# clean test data
model2.evaluate(X_test, y_test)

[2.1281369037628175, 0.5127]

In [ ]:
# assistive test data
model2.evaluate(X_test_assistive, y_test)

[0.11127971088027117, 0.9695]

In [ ]:
# adversarial test data
model2.evaluate(X_test_adv, y_test)

[10.301456629943848, 0.1218]

# Assistive Signals入り画像とクリーン画像を5:5で学習

In [ ]:
X_train_half = list(X_train[:25000])
X_train_half.extend(X_train_assistive[25000:])
X_train_half = np.array(X_train_half)

In [ ]:
model3 = build_model()

In [ ]:
model3.fit(X_train_half, y_train,
          batch_size=512,
          epochs=30,
          shuffle=True)

Train on 50000 samples
Epoch 1/30
50000/50000 [==============================] - 30s 600us/sample - loss: 2.1470 - accuracy: 0.1721
Epoch 2/30
50000/50000 [==============================] - 30s 597us/sample - loss: 1.7743 - accuracy: 0.3291
Epoch 3/30
50000/50000 [==============================] - 30s 596us/sample - loss: 1.4012 - accuracy: 0.4791
Epoch 4/30
50000/50000 [==============================] - 30s 596us/sample - loss: 1.1264 - accuracy: 0.5957
Epoch 5/30
50000/50000 [==============================] - 30s 596us/sample - loss: 0.9157 - accuracy: 0.6784
Epoch 6/30
50000/50000 [==============================] - 30s 598us/sample - loss: 0.7605 - accuracy: 0.7355
Epoch 7/30
50000/50000 [==============================] - 30s 598us/sample - loss: 0.6765 - accuracy: 0.7665
Epoch 8/30
50000/50000 [==============================] - 30s 597us/sample - loss: 0.5899 - accuracy: 0.7951
Epoch 9/30
50000/50000 [==============================] - 30s 597us/sample - loss: 0.5079 - accuracy: 0.8

In [ ]:
# assistive train data
model3.evaluate(X_train_assistive, y_train)

In [ ]:
# clean test data
model3.evaluate(X_test, y_test)

In [ ]:
# assistive test data
model3.evaluate(X_test_assistive, y_test)

In [ ]:
# adversarial test data
model3.evaluate(X_test_adv, y_test)

# 半分のデータ数で学習

In [ ]:
X_train_half = X_train[:25000]
y_train_half = y_train[:25000]

X_train_assistive_half = X_train_assistive[:25000]

In [ ]:
model4_clean = build_model()
model4_assistive = build_model()

In [ ]:
model4_clean.fit(x=X_train_half, y=y_train_half,
                 epochs=30,
                 batch_size=512,
                 shuffle=True)

In [ ]:
model4_assistive.fit(x=X_train_assistive_half, y=y_train_half,
                 epochs=30,
                 batch_size=512,
                 validation_data=(X_test, y_test),
                 shuffle=True)

In [ ]:
model4_clean.evaluate(X_test, y_test)

In [ ]:
model4_assistive.evaluate(X_test, y_test)